# 第3章 字典和集合

## 映射类型

In [1]:
from collections import abc, OrderedDict, UserDict

# isinstance和type区别
# isinstance()：认为子类是一种父类类型，考虑继承关系
# type()：不会认为子类是一种父类类型，不考虑继承关系。
# 如果要判断两个类型是否相同推荐使用 isinstance()。
my_dict = {}
print(isinstance(my_dict, abc.Mapping), type(my_dict).__name__ == 'dict')
my_dict2 = OrderedDict([])
print(isinstance(my_dict2, abc.Mapping), type(my_dict2).__name__ == 'dict')


class StrkeyDict(UserDict):
    pass


my_dict3 = StrkeyDict()
print(isinstance(my_dict3, abc.Mapping), type(my_dict2).__name__ == 'dict')


True True
True False
True False


In [2]:
# 散列表
tt = (1, 2, (5, 4))
hash(tt)

-2764265307498484889

In [3]:
t1 = (1, 2, (5, 4))
tt == t1

True

In [4]:
hash(t1)

-2764265307498484889

In [5]:
# 可变的元组＋不可变的列表＝不可散列
# 不可变不是充要条件
t2 = (1, 2, [5, 4])
hash(t2)

TypeError: unhashable type: 'list'

In [9]:
t3 = (1, 2, frozenset([30, 40]))
print(hash(t3))

5149391500123939311


In [11]:
# 原子：数值、str、bytes不可变类型，属于可散列类型
hash(1)

1

## 字典创建与推导
字典推导（dictcomp）可以从任何以键值对作为元素的可迭代对象中构建出字典

### 字典的创建

In [18]:
# 字典的创造
a = dict(one=1, two=2, three=3)
b = {'one': 1, "two": 2, "three": 3}
c = dict(zip(["one", "two", "three"], [1, 2, 3]))  # 拉链函数:可迭代对象 打包成元组
d = dict([("two", 2), ('one', 1), ("three", 3)])  #
e = dict({"two": 2, 'one': 1, "three": 3})

print(a == b == c == e == d)

True


In [21]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [4, 5, 6, 7, 8]
zipped = zip(a, b)  # 打包为元组的列表

In [23]:
print(type(zipped))
print(*zipped)  # 与 zip 相反，*zipped 可理解为解压，返回二维矩阵式
print(*zip(a, c))  # 元素个数与最短的列表一致

<class 'zip'>

(1, 4) (2, 5) (3, 6)


In [25]:
# 拆包
[*zip(a, c)]

[(1, 4), (2, 5), (3, 6)]

In [15]:
# zip在for循环中的使用
for i,j in zip(a, c):
    print(i, j)

1 4
2 5
3 6


### 字典的推导

In [3]:
dial_codes = [(86, 'China'),
              (91, 'India'),
              (1, 'United States'),
              (62, 'Indonesia'),
              (55, 'Brazil'),
              (92, 'Pakistan'),
              (880, 'Bangladesh'),
              (234, 'Nigeria'),
              (7, 'Russia'),
              (81, 'Japan')]

print(dial_codes)

[(86, 'China'), (91, 'India'), (1, 'United States'), (62, 'Indonesia'), (55, 'Brazil'), (92, 'Pakistan'), (880, 'Bangladesh'), (234, 'Nigeria'), (7, 'Russia'), (81, 'Japan')]


In [4]:
# 利用推导来获取数据
country_code = {country: code for code, country in dial_codes}
print(country_code)

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62, 'Brazil': 55, 'Pakistan': 92, 'Bangladesh': 880, 'Nigeria': 234, 'Russia': 7, 'Japan': 81}


In [5]:
# 对字典创建进行判断
country_code2 = {code: country.upper() for code, country in dial_codes  if code < 66}
print(country_code2)

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}


### 鸭子类型
核心要点：你既可以用一个映射对象来新建一个映射对象，也可以用包含(key, value)元素的可迭代对象来初始化一个映射对象。

In [33]:
# 鸭子类型的更新方法
# 含有keys方法
item.update({"d":5})
# iterable
item.update([("a", 1), ("b", 2)])
item

{'d': 5, 'a': 1, 'b': 2}


## 常见的映射方法

### 键值缺失情况
一般步骤：  
(1)找到缺失key  
(2)填补缺失value  
(3)重新映射key-->value的关系

In [20]:
#　当键值不存在时
item = {}
item["python"]

KeyError: 'python'

In [21]:
# 后处理不存在值
values = item.get("python", [])
values

[]

In [22]:
# 添加步骤
# 两步查找，先查找取出默认，后查找放入取值
values.append(5)
item["python"] = values
item

{'python': [5]}

In [24]:
# 设置字典的默认属性，插入值
item = {}
item.setdefault("python", [])
item["python"]

[]

### 特殊方法__mission__
核心要点：所有的映射类型在处理找不到的键的时候，都会牵扯到__missing__方法。这也是这个方法称作“missing”的原因。虽然基类dict并没有定义这个方法，但是dict是知道有这么个东西存在的。也就是说，如果有一个类继承了dict，然后这个继承类提供了__missing__方法，那么在__getitem__碰到找不到的键的时候，Python就会自动调用它，而不是抛出一个KeyError异常。

In [147]:
"""
查询的时候，映射类型里面的键，统统换成str
如果一个类例如StrKeyDict0继承了dict。然后这个继承类提供了一个__missing__方法，
那么在__getitem__碰到找不到的键的时候，Python会自动调用它
"""


class StrKeyDict0(dict):
    # 该类继承了dict
    def __missing__(self, key):
        print("i'm missing")
        # 如果找不到的键，本身就是字符串，抛异常
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def get(self, key, default=None):
        try:
            return self[key]  # 通过这种形式委托给__getitem__.如果这个时候再找不到那就走__missing__
        except KeyError:
            return default  # 如果__missing__也是失败

    def __contains__(self, key):
        # 先按照传入键的原本的值来查找（我们的映射类型中可能含有非字符串的键），如果没 找到，再用 str() 方法把键转换成字符串再查找一次。
        return key in self.keys() or str(key) in self.keys()

In [148]:
# 当有非字符串的键被查找的时候，StrKeyDict0 是如何在该键不存在的情况下， 把它转换为字符串的
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
# 一次missing方法
print(d['2'])
print("-------非字符串查找,转换成字符串后存在--------")
# 两次missing方法
print(d[4])
print("-------非字符串查找,转换成字符串后仍然不存在--------")
print(d[1])

two
-------非字符串查找,转换成字符串后存在--------
i'm missing
four
-------非字符串查找,转换成字符串后仍然不存在--------
i'm missing
i'm missing


KeyError: '1'

In [149]:
d.get('2')
d.get(4)
# get方法不存在，则以默认值的形式表示
print(d.get(1))
# __contains__方法
print( 2 in d )
print( 1 in d )

i'm missing
i'm missing
i'm missing
None
True
False


### 默认字典 defaultdict

In [37]:
# 缺失字典的更新
import sys
import re

WORD_RE = re.compile(r'\w+')
index = {}
with open(sys.argv[0], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            # 方法一
            # occurrences = index.get(word, []) # 提取 word 出现的情况，如果还没有它的记录返回 []
            # occurrences.append(location) # 把新单词出现的位置添加到列表的后面
            # index[word] = occurrences # 把新的列表放回到字典中 #又涉及一次到查询
            # 方法二
            # 如果word值第一次出现，则为[],否则追加元素
            index.setdefault(word,[]).append(location)
            
# 以字母顺序打印结果
for word in sorted(index, key=str.upper):
    print(word, index[word])

0 [(12, 17), (13, 22)]
added [(11, 15)]
after [(4, 1)]
an [(1, 30)]
app [(15, 40), (16, 5)]
as [(15, 37)]
avoid [(3, 55)]
back [(11, 21)]
by [(11, 26)]
can [(3, 51)]
cwd [(4, 20)]
CWD [(10, 18)]
del [(13, 9)]
doing [(3, 61)]
Entry [(1, 4)]
for [(1, 16)]
from [(3, 18), (4, 24), (10, 22), (15, 5)]
if [(9, 1), (12, 5)]
import [(7, 1), (15, 20)]
imports [(3, 67)]
init_path [(11, 49)]
InteractiveShellApp [(11, 29)]
ipykernel [(3, 27), (15, 10)]
IPython [(1, 33)]
is [(3, 6), (11, 12)]
kernel [(1, 41)]
kernelapp [(15, 27)]
launching [(1, 20)]
launch_new_instance [(16, 9)]
load [(10, 45)]
package [(3, 37)]
path [(4, 33), (10, 31), (12, 12), (13, 17)]
point [(1, 10)]
Remove [(10, 7)]
removing [(4, 7)]
separate [(3, 9)]
so [(3, 45)]
stuff [(10, 50)]
sys [(4, 29), (7, 8), (10, 27), (12, 8), (13, 13)]
the [(3, 23), (4, 16), (10, 14)]
This [(3, 1), (11, 7)]
until [(3, 75)]
we [(3, 48), (10, 42)]
while [(10, 36)]
__main__ [(9, 17)]
__name__ [(9, 4)]


In [26]:
from collections import defaultdict
# 前处理不存在值
# 默认字典保证键有值
#参数是类型
# 默认字典，一致性：所有字典类型表示为同一类型,可以使用相应类型下的方法
# 例如:当key值存在，而无value值时，会默认生成{key:默认类型}
d=defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d['a'].append(2)
d['b'].append(3)
# 只有键，默认[]
d["c"]
print(d)

defaultdict(<class 'list'>, {'a': [1, 2, 2], 'b': [3], 'c': []})


In [32]:
a = defaultdict(set)

In [33]:
a["1"]

set()

In [39]:
d2=defaultdict(set)
d2['a'].add(1)
d2['a'].add(2)
d2['a'].add(2)
d2['b'].add(3)
d2["c"]
print(d2)

defaultdict(<class 'set'>, {'a': {1, 2}, 'b': {3}, 'c': set()})


In [43]:
d3 = {}
d3.setdefault('a', []).append(1)
d3.setdefault('a', []).append(2)
d3.setdefault('b', []).append(3)
print(d3)  # {'a': [1, 2], 'b': [3]}

{'a': [1, 2], 'b': [3]}


In [45]:
# 默认在value不存在的情况下，用整数0进行补充
d4 = defaultdict(int)
d4["4"] = 1
d4["5"]
d4

defaultdict(int, {'4': 1, '5': 0})

In [48]:
# 普通字典类型
d5 = {}
# 增加默认字典类型
d5.setdefault("a", [])
d5.setdefault("b", set())
d5["a"].append(1)
d5["a"].append(1)
d5["b"].add(88)
d5["b"].add(8)
d5["b"].add(8)
d5

{'a': [1, 1], 'b': {8, 88}}

### 有序字典 OrderedDict
核心要点：一个 OrderedDict 的大小是一个普通字典的两倍，因为它内部维护着另外一个链表OrderedDict 内部维护着一个根据键插入顺序排序的双向链表。每次当一个新的元素插入进来的时候，它会被放到链表的尾部。对于一个已经存在的键的重复赋值不会 改变键的顺序

In [49]:
from collections import OrderedDict

d4 = OrderedDict()
d4['a'] = 1
d4['b'] = 2
d4['c'] = 3
d4['d'] = 4
for k, v in d4.items():
    print(k, '=====', v)

a ===== 1
b ===== 2
c ===== 3
d ===== 4


In [50]:
import json
print(json.dumps(d4)) # {"a": 1, "b": 2, "c": 3, "d": 4}

{"a": 1, "b": 2, "c": 3, "d": 4}


In [51]:
# 删除
d4.popitem() #默认一处字典中最先插入的元素（先进后出）
print(json.dumps(d4)) # {"a": 1, "b": 2, "c": 3}

{"a": 1, "b": 2, "c": 3}


In [52]:
d4.popitem(last=False) #默认一处字典中最先插入的元素（先进先出）
print(json.dumps(d4)) # {"b": 2, "c": 3}

{"b": 2, "c": 3}


In [53]:
print("-----循环-----")
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}
price1 = zip(prices.values(), prices.keys())
for k, v in price1.__iter__():
    print(k, '==', v)

-----循环-----
45.23 == ACME
612.78 == AAPL
205.55 == IBM
37.2 == HPQ
10.75 == FB


In [58]:
# 普通字典无序，但按照存储顺序读写
print("------普通字典-------")
dicta = {"a": 1, "b": 2}
dictb = {"b": 2, "a": 1, }
print(dicta, dictb)
print(dicta == dictb)

------普通字典-------
{'a': 1, 'b': 2} {'b': 2, 'a': 1}
True


In [55]:
# 列表有序
print("------列表-------")
lista = ["a", "b"]
listb = ["b", "a", ]
print(lista, listb)
print(lista == listb)

------列表-------
['a', 'b'] ['b', 'a']
False


In [56]:
print("------有序字典-------")
from collections import OrderedDict

d4 = OrderedDict()
d4['a'] = 1
d4['b'] = 2
d5 = OrderedDict()
d5['b'] = 2
d5['a'] = 1
print(d4, d5)
print(d4 == d5)

------有序字典-------
OrderedDict([('a', 1), ('b', 2)]) OrderedDict([('b', 2), ('a', 1)])
False


### 链映射ChainMap
核心要点：该类型可以容纳数个不同的映射对象，然后在进行键查找操作的时候，这些对象会被当作一个整体被逐个查找，直到键被找到为止。因此，ChainMap实际上是把放入的字典存储在一个队列中，<font color = "red">当进行字典的增加删除等操作只会在第一个字典上进行，当进行查找的时候会依次查找。</font>

In [59]:
from collections import ChainMap

a = {"x": 1, "z": 3}
b = {"y": 2, "z": 4}

c = ChainMap(a, b)
print(c)

ChainMap({'x': 1, 'z': 3}, {'y': 2, 'z': 4})


In [60]:
# 查找操作：依次查找，首次查找之后，停止继续查找
c["z"], c["y"]

(3, 2)

In [62]:
# 对象的引用
a.update({"z": 2})
print(c)  # 同步更新

ChainMap({'x': 1, 'z': 2}, {'y': 2, 'z': 4})


In [66]:
# 当对ChainMap进行修改的时候总是只会对第一个字典进行修改
print(c["z"])
c["z"]=5
print(c)
a

5
ChainMap({'x': 1, 'z': 5}, {'y': 2, 'z': 4})


{'x': 1, 'z': 5}

In [68]:
# 增加 只对第一个字典进行修改，即使key在后面的字典中存在
c["y"] = 100
c

ChainMap({'x': 1, 'z': 5, 'y': 100}, {'y': 2, 'z': 4})

In [69]:
# 删除
c.pop('z')
print(c)

ChainMap({'x': 1, 'y': 100}, {'y': 2, 'z': 4})


In [74]:
c.pop('z')  # 报错
del c["z"]    # 报错

KeyError: "Key not found in the first mapping: 'z'"

In [75]:
print('-------------------')
a = ChainMap()
a["x"]=1
print(a) # ChainMap({'x': 1})

-------------------
ChainMap({'x': 1})


In [80]:
b=a.new_child()
b["4"] = 4
a, b

(ChainMap({'x': 1}), ChainMap({'4': 4}, {'x': 1}))

### 计数器Counter
核心要点: 这个映射类型会给键准备一个整数计数器。每次更新一个键的时候都会增加这个计数器。所以这个类型可以用来给可散列表对象计数，或者是当成多重集来用——多重集合就是集合里的元素可以出现不止一次。

In [34]:
from collections import Counter

In [35]:
s = 'To be,or not to be:that is the question'
# Counter存储为键值对的形式，因此按照字典的形式进行处理
c = Counter(s)
print(c)

Counter({' ': 7, 't': 6, 'o': 5, 'e': 4, 'b': 2, 'n': 2, 'h': 2, 'i': 2, 's': 2, 'T': 1, ',': 1, 'r': 1, ':': 1, 'a': 1, 'q': 1, 'u': 1})


In [36]:
#Counter({' ': 7, 't': 6, 'o': 5, 'e': 4, 'b': 2, 'n': 2, 'h': 2, 'i': 2, 's': 2, 'T': 1, ',': 1, 'r': 1, ':': 1, 'a': 1, 'q': 1, 'u': 1})
total = sum([v for k, v in c.items() if k in ['y', 'a', 'n', 'g', 'o']])
print(total) # 简单理解c是一个dict，使用items()方法可以遍历dict的key和value。 如果 k

8


In [37]:
# 同样的，可以对list统计
elements = ['jack', 'JACK', 'lucy', 'April', 'Lucy', 'mike', 'JAck']
c = Counter(elements)
print(c) # Counter({'jack': 1, 'JACK': 1, 'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'JAck': 1})

Counter({'jack': 1, 'JACK': 1, 'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'JAck': 1})


In [38]:
# 众数
c.most_common(3)

[('jack', 1), ('JACK', 1), ('lucy', 1)]

In [39]:
# 更新子键值
new_elements=['alice','steven']
c.update(new_elements)
print(c) # Counter({'jack': 1, 'JACK': 1, 'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'JAck': 1, 'alice': 1, 'steven': 1})

Counter({'jack': 1, 'JACK': 1, 'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'JAck': 1, 'alice': 1, 'steven': 1})


# 消除子键值
new_elements=['JACK','JAck','jaCK']
c.subtract(new_elements)
print(c) # Counter({'jack': 1, 'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'alice': 1, 'steven': 1, 'JACK': 0, 'JAck': 0, 'jaCK': -1})

In [92]:
del c['JACK']
print(c) # Counter({'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'alice': 1, 'steven': 1, 'JACK': 0, 'JAck': 0, 'jaCK': -1})

Counter({'lucy': 1, 'April': 1, 'Lucy': 1, 'mike': 1, 'alice': 1, 'steven': 1, 'JAck': 0, 'jaCK': -1})


In [94]:
a = Counter('aaab')
b = Counter('ccdd')

In [95]:
a, b

(Counter({'a': 3, 'b': 1}), Counter({'c': 2, 'd': 2}))

In [96]:
# + 和
a_sum_b = a + b
print(a_sum_b)

Counter({'a': 3, 'c': 2, 'd': 2, 'b': 1})


In [97]:
# - 差
a_sub_b = a - b
print(a_sub_b)

Counter({'a': 3, 'b': 1})


In [98]:
# & 交集
a_and_b = a & b
print(a_and_b)

Counter()


In [99]:
# | 并集
a_union_b = a | b
print(a_union_b)

Counter({'a': 3, 'c': 2, 'd': 2, 'b': 1})


### 子类化 UserDict

In [232]:
from collections import UserDict


class StrkeyDict(UserDict):
    def __missing__(self, key):
        if isinstance(key,str):
            raise KeyError
        return self[str(key)]

    def __setitem__(self, key, value):
        self.data[str(key)]=value

    def __contains__(self, key):
        return str(key) in self.data

In [236]:
# 当有非字符串的键被查找的时候，StrkeyDict 是如何在该键不存在的情况下， 把它转换为字符串的
d = StrkeyDict([('2', 'two'), ('4', 'four')])
print(d['2'])
print("-------非字符串查找,转换成字符串后存在--------")
print(d[4])
print("-------非字符串查找,转换成字符串后仍然不存在--------")
# print(d[1]) # 报错

two
-------非字符串查找,转换成字符串后存在--------
four
-------非字符串查找,转换成字符串后仍然不存在--------


In [240]:
d.get('2')

'two'

In [241]:
d.get(4)

'four'

In [239]:
# 返回默认值None
print(d.get(1))

None


In [237]:
print( 2 in d )
print( 1 in d )

None
True
False


## 不可变的映射类型（只读）
问题来源: 标准库里所有的映射类型都是可变的，但有时候你会有这样的需求，比如不能让用户错误地修改某个映射。  
核心要点：如果给这个类一个映射，它会返回一个只读的映射视图。虽然是个只读视图，但是它是<font color=red>动态的</font>。这意味着如果对原映射做出了改动，我们通过这个视图可以观察到，但是无法通过这个视图对原映射做出修改。

In [102]:
from types import MappingProxyType

d = {1: 'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)  # {1: 'A'}
print(d_proxy[1])  # A

{1: 'A'}
A


In [103]:
# 改变原始映射, 其对应的视图也进行变化(动态的)
# 因此只能通过原始映射进行修改
d[2] = "B"
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [106]:
# 对视图不能直接进行修改，即映射后的视图属于不可修改类型
d_proxy[2] = 'x' #  直接赋值就报错

TypeError: 'mappingproxy' object does not support item assignment

## 集合
“散列表（Hash table，也叫哈希表），是根据关键码值(Key value)而直接进行访问的数据结构。也就是说，它通过把关键码值映射到表中一个位置来访问记录，以加快查找的速度。这个映射函数叫做散列函数，存放记录的数组叫做散列表。给定表M，存在函数f(key)，对任意给定的关键字值key，代入函数后若能得到包含该关键字的记录在表中的地址，则称表M为哈希(Hash）表，函数f(key)为哈希(Hash) 函数。”

In [109]:
# 空集合创建的误区
a = {}  # 默认为字典类型，因此，如果是创建空集合，必须是set()
b = set()  # 创建空集合
c = {1}  # 包含集合类型的元素时为集合
d = {2: "B"}  # 包含字典类型的元素时为集合

type(a), type(b), type(c), type(d)

(dict, set, set, dict)

In [108]:
# 集合属于不可散列的
hash(b)

TypeError: unhashable type: 'set'

In [282]:
a = {1, 2, 3}
b = {3, 4, 5}

In [283]:
# 交集  s &= z
print(a & b, a, b)  # {3} {1, 2, 3} {3, 4, 5}
print(b & a, a, b)

{3} {1, 2, 3} {3, 4, 5}
{3} {1, 2, 3} {3, 4, 5}


In [284]:
# 差集  s -= z
print(a - b, a, b)  # {1, 2} {1, 2, 3} {3, 4, 5}
print(b - a, a, b)  # {4, 5} {1, 2, 3} {3, 4, 5}

{1, 2} {1, 2, 3} {3, 4, 5}
{4, 5} {1, 2, 3} {3, 4, 5}


In [285]:
# 并集  s |= z
print(a | b, a, b)  # {1, 2, 3, 4, 5, 6} {1, 2, 3} {3, 4, 5}
print(b | a, a, b)

{1, 2, 3, 4, 5} {1, 2, 3} {3, 4, 5}
{1, 2, 3, 4, 5} {1, 2, 3} {3, 4, 5}


In [286]:
# 补集  s ^= z
print(a ^ b, a, b)  # {1, 2, 4, 5} {1, 2, 3} {3, 4, 5}
print(b ^ a, a, b)  # {1, 2, 4, 5} {1, 2, 3} {3, 4, 5}

{1, 2, 4, 5} {1, 2, 3} {3, 4, 5}
{1, 2, 4, 5} {1, 2, 3} {3, 4, 5}


In [287]:
# 集合之间的运算
# <= 子集（包含本身）, < 真子集（不包含本身）
c = {1, 2}
e = 2
f = {1, 2, 3}
print("-----比较运算符-----")
print(e in a)  # 元素 e 是否属于 s
print(f<=a,c<=a)
print(f<a,c<a)

-----比较运算符-----
True
True True
False True


## dict和set的背后

In [288]:
"""
整体思想其实并不难
1：主要功能包含：存值、取值。
2：丰富：存值得时候调用了hash算法，随便处理了hash冲突
3：hash算法：用的是 取余算法
4：解决冲突：用的是
"""


class HashTable:
    def __init__(self):
        self.size = 11  # hash szie
        self.slots = [None] * self.size  # 插槽
        self.data = [None] * self.size  # 数据

    def put(self, key, data):
        # 通过 hash 算法，计算出插槽的位置
        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] == None:
            # 如果 这个插槽为空
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            # 计算后的hash值和key值相等
            # eg: [54] == "A", [54] == "B" 
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  # replace
            else:
                # hash 冲突 通过 重新计算hash值的办法 继续
                # [54] = "A", [44] = "B", hash_value = 10
                nextslot = self.rehash(hashvalue, len(self.slots))
                while self.slots[nextslot] != None and self.slots[nextslot] != key:
                    nextslot = self.rehash(nextslot, len(self.slots))

                if self.slots[nextslot] == None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                else:
                    self.data[nextslot] = data  # replace

    def hashfunction(self, key, size):
        """哈希方法，取余"""
        return key % size

    def rehash(self, oldhash, size):
        """解决冲突，重新开地址的方法"""
        return (oldhash + 1) % size

    def get(self, key):
        # 获取的时候也一样，
        startslot = self.hashfunction(key, len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        while self.slots[position] != None and not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.rehash(position, len(self.slots))
                if position == startslot:
                    stop = True
        return data

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

In [289]:
H = HashTable()     # hash value
H[54] = "cat"       # 10
H[26] = "dog"       # 4
H[93] = "lion"      # 5
H[17] = "tiger"     # 6
H[77] = "bird"      # 0
H[31] = "cow"       # 10
H[44] = "goat"      # 0
H[55] = "pig"       # 0
H[20] = "chicken"   # 9
print(H.slots)
print(H.data)
print(H[20])1
print(H[17])
H[20] = 'duck'
print(H[20])
print(H[99])

[77, 44, 55, 20, 26, 93, 17, None, None, 31, 54]
['bird', 'goat', 'pig', 'chicken', 'dog', 'lion', 'tiger', None, None, 'cow', 'cat']
chicken
tiger
duck
None


## 本章小结
- 字典算得上是Python的基石。除了基本的dict之外，标准库还提供现成且好用的特殊映射类型，比如defaultdict、OrderedDict、ChainMap和Counter。这些映射类型都属于collections模块，这个模块还提供了便于扩展的UserDict类。  
- 大多数映射类型都提供了两个很强大的方法：setdefault和update。setdefault方法可以用来更新字典里存放的可变值（比如列表），从而避免了重复的键搜索。update方法则让批量更新成为可能，它可以用来插入新值或者更新已有键值对，它的参数可以是包含(key, value)这种键值对的可迭代对象，或者关键字参数。映射类型的构造方法也会利用update方法来让用户可以使用别的映射对象、可迭代对象或者关键字参数来创建新对象。　　
- 在映射类型的API中，有个很好用的方法是__missing__，当对象找不到某个键的时候，可以通过这个方法自定义会发生什么。　　
- collections.abc模块提供了Mapping和MutableMapping这两个抽象基类，利用它们，我们可以进行类型查询或者引用。不太为人所知的MappingProxyType可以用来创建不可变映射对象，它被封装在types模块中。另外还有Set和MutableSet这两个抽象基类。　　
- dict和set背后的散列表效率很高，对它的了解越深入，就越能理解为什么被保存的元素会呈现出不同的顺序，以及已有的元素顺序会发生变化的原因。同时，速度是以牺牲空间为代价而换来的。

## 参考
[python collections模块详解](https://www.cnblogs.com/dahu-daqing/p/7040490.html)  
[流畅的Python](https://weread.qq.com/web/reader/ab832620715c017eab864a6)